In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import re
from collections import defaultdict

In [2]:
# DA = 'data analyst'
# DS = 'data scientist'
# DE = 'data engineer'

# South Africa
sa_DA = pd.read_csv("sa_DA.csv")
sa_DS = pd.read_csv("sa_DS.csv")
sa_DE = pd.read_csv("sa_DE.csv")

# United States
us_DA = pd.read_csv("us_DA.csv")
us_DS = pd.read_csv("us_DS.csv")
us_DE = pd.read_csv("us_DE.csv")

# United Kingdom
uk_DA = pd.read_csv("uk_DA.csv")
uk_DS = pd.read_csv("uk_DS.csv")
uk_DE = pd.read_csv("uk_DE.csv")

# Australia
au_DA = pd.read_csv("au_DA.csv")
au_DS = pd.read_csv("au_DS.csv")
au_DE = pd.read_csv("au_DE.csv")

dataframes = [sa_DA, sa_DS, sa_DE,
              us_DA, us_DS, us_DE,
              uk_DA, uk_DS, uk_DE,
              au_DA, au_DS, au_DE]

In [3]:
sa_DA.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Transformation Analyst,-1,Whats in it for you?\n\nThis position offers n...,4.0,IHS Markit\n4.0,Cape Town,-1,10000+ Employees,2016,Company - Public,Consulting,Business Services,$2 to $5 billion (USD),-1
1,Data Analyst,-1,Minimum requirements for the role:\n\nA degree...,3.1,Prostaff Holdings\n3.1,Johannesburg,-1,Unknown,-1,Unknown,-1,-1,Less than $1 million (USD),-1
2,Reporting & Data Analyst,-1,Reporting & Data Analyst\n\nIntroduction:\n\nA...,2.9,The Revenue Optimization Companies\n2.9,Remote,-1,5001 to 10000 Employees,2000,Company - Private,Consumer Electronics & Appliances Stores,Retail,$100 to $500 million (USD),-1
3,Junior Data Developer System Analyst,-1,Adapt IT\nJunior Data Developer System Analyst...,4.0,AdaptIT\n4.0,Johannesburg,-1,51 to 200 Employees,-1,Company - Private,-1,-1,Less than $1 million (USD),-1
4,"Data Analyst - Johannesburg, South Africa",-1,EY’s newly established Africa Data Delivery Ce...,3.8,EY\n3.8,Johannesburg,-1,10000+ Employees,1989,Company - Private,Accounting,Accounting & Legal,$10+ billion (USD),-1


In [4]:
sa_DA.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [5]:
# unfortunately the data from a glassdoor search quuery has duplicate job listings
# need to drop duplictate jobs: (job description is the best identifier)

counter = 0
for df in dataframes:
    if counter == 0: print("SA") 
    if counter == 3: print("\nUS")
    if counter == 6: print("\nUK")
    if counter == 9: print("\nAU")
    counter += 1
    print(str(counter)+": ",len(df["Job Description"]),"(original)",
          len(df.drop_duplicates(subset="Job Description")),"(dropped)")

SA
1:  78 (original) 76 (dropped)
2:  220 (original) 214 (dropped)
3:  790 (original) 709 (dropped)

US
4:  1000 (original) 720 (dropped)
5:  1000 (original) 673 (dropped)
6:  800 (original) 551 (dropped)

UK
7:  785 (original) 646 (dropped)
8:  1000 (original) 699 (dropped)
9:  800 (original) 622 (dropped)

AU
10:  1000 (original) 160 (dropped)
11:  468 (original) 168 (dropped)
12:  800 (original) 157 (dropped)


In [6]:
for df in dataframes:
    df.drop_duplicates(subset="Job Description", inplace=True)

In [7]:
# remove new line chars from job descriptions, company name

for df in dataframes:
    df["Job Description"] = df["Job Description"].str.replace("\n", " ")
    df["Company Name"] = df["Company Name"].str.replace("\n", " ")

In [8]:
da_ds_de_techs = ["Python", "R", "Julia", "Java", "C", "Scala", "Bash", "PowerBi", "Tableau", "Excel", "VBA", "Macros","Alteryx", 
         "Spotfire", "Qlik", "Looker", "SAS", "ERP", "SAP", "MS Navision", "SAGE", "AccPac", "SQL", "Sql", "NoSql", "NoSQL", 
         "MongoDB", "Postgres", "PostgreSQL", "mySQL", "XML", "HTML", "CSS", "Javascript", "Hadoop", "Spark", "PySpark", 
         "SparkR", "Kafka", "ETL", "Oracle", "Storm", "AWS", "Cassandra", "Azkaban", "Airflow", "Luigi", "MATLAB", 
         "Snowflake", "BigQuery", "Azure", "GCP", "Perl", "Presto", "HBase", "Hive", "Druid", "Docker", "TensorFlow"]
pattern = "|".join(r"\b{}\b".format(x) for x in da_ds_de_techs)

In [9]:
for df in dataframes:
    df["Technologies"] = df["Job Description"].str.findall(pattern, flags=re.I).str.join(", ").replace("", "Not found")

In [10]:
sa_DA["Technologies"].head()

0                                                  VBA
1                                            Not found
2      Tableau, PowerBI, SQL, R, Python, Excel, macros
3                        Oracle, SQL, Tableau, PowerBI
4    macros, SQL, XML, Javascript, ETL, ERP, SAP, O...
Name: Technologies, dtype: object

In [11]:
# applying the above method which returns a series matching all items in the list; will need two lists based on below.

In [12]:
da_ds_de_techs = ["quantititive", "IT", "engineering", "marketing", "business", "commerce", 
                    "statistics", "maths", "mathematics", "physics", "chemistry", "epidemiology"]
pattern = "|".join(r"\b{}\b".format(x) for x in da_ds_de_techs)

In [13]:
for df in dataframes:
    df["Technologies"] = df["Job Description"].str.findall(pattern, flags=re.I).str.join(", ").replace("", "Not found")

In [14]:
da_ds_de_techs = ["computer", "data"]
pattern = "|".join(r"\b{}\b".format(x) for x in da_ds_de_techs)

In [15]:
for df in dataframes:
    df["Technologies"] = df["Job Description"].str.findall(pattern, flags=re.I).str.join(", ").replace("", "Not found")

In [16]:
sa_DA["Job Description"]

0     Whats in it for you?  This position offers not...
1     Minimum requirements for the role:  A degree i...
2     Reporting & Data Analyst  Introduction:  At T-...
3     Adapt IT Junior Data Developer System Analyst-...
4     EY’s newly established Africa Data Delivery Ce...
                            ...                        
73    Northern Suburbs. R55k. Require person with a ...
74    Company Description  WNS South Africa is a lea...
75    Description  Looking to make a difference and ...
76    You are an all-round “techie” with strong inte...
77    ENVIRONMENT:  Minimum of 4 years’ in software/...
Name: Job Description, Length: 76, dtype: object

In [17]:
def get_degree_level(df):
    """
    Degree level
    """
 
    result = []

    for row in df["Job Description"]:
        try:
            jd = row.split(" ")
            
            if "bachelors" in map(lambda x: x.lower(), jd):
                result.append(["bachelors"])
            elif "Bachelors" in map(lambda x: x.lower(), jd):
                result.append(["bachelors"])
            elif "bachelor" in map(lambda x: x.lower(), jd):
                result.append(["bachelors"])
            elif "Bachelor" in map(lambda x: x.lower(), jd):
                result.append(["bachelors"])
            elif "bachelor's" in map(lambda x: x.lower(), jd):
                result.append(["bachelors"])
            elif "Bachelor's" in map(lambda x: x.lower(), jd):
                result.append(["bachelors"])
            if "master's" in map(lambda x: x.lower(), jd):   
                result.append(["masters"])
            elif "Master's" in map(lambda x: x.lower(), jd):                       
                result.append(["masters"])
            elif "masters" in map(lambda x: x.lower(), jd):                       
                result.append(["masters"])
            elif "Masters" in map(lambda x: x.lower(), jd):                       
                result.append(["masters"])
            if "phd" in map(lambda x: x.lower(), jd):
                result.append(["phd"])
            elif "Phd" in map(lambda x: x.lower(), jd):
                result.append(["phd"])
            elif "Ph.D" in map(lambda x: x.lower(), jd):
                result.append(["phd"])
            elif "PH.D" in map(lambda x: x.lower(), jd):
                result.append(["phd"])
            elif "doctorate" in map(lambda x: x.lower(), jd):
                result.append(["phd"])

            else: 
                result.append(["UNSPECIFIED"])
                continue
                
        except: continue
        
    return result

In [18]:
degree_level = get_degree_level(us_DE)

In [19]:
degree_level

[['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['masters'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['masters'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['masters'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['UNSPECIFIED'],
 ['bachelors'],
 ['UNSPECIFIED'],
 ['UNSPECIFI

In [20]:
def get_degree_major(df):
    """
    Degree type (major)
    """
    
    # add to this list
    majors_short = ["quantititive", "IT", "engineering", "marketing", "business", "commerce", 
                    "statistics", "maths", "mathematics", "physics", "chemistry", "epidemiology"]
    VALIDATE = ["engineering", "mathematics", "actuarial"]
    majors_long = ["computer", "data"]
    DTEST = defaultdict(int)
    result = []
    
    for row in df["Job Description"]:
        try:
            if "degree" in row.lower():
                subset = row.split("degree")[1]
                for major in majors_short:
                    if major in subset:
                        result.append(major)
        
        except: continue
            
    return result

In [21]:
degree_type = get_degree_major(us_DS)

In [22]:
degree_type

['statistics',
 'mathematics',
 'IT',
 'marketing',
 'marketing',
 'business',
 'commerce',
 'business',
 'engineering',
 'business',
 'business',
 'commerce',
 'business',
 'IT',
 'statistics',
 'mathematics',
 'physics',
 'chemistry',
 'business',
 'statistics',
 'mathematics',
 'IT',
 'engineering',
 'statistics',
 'statistics',
 'statistics',
 'mathematics',
 'engineering',
 'statistics',
 'business',
 'business',
 'engineering',
 'business',
 'engineering',
 'business',
 'engineering',
 'mathematics',
 'IT',
 'engineering',
 'business',
 'business',
 'mathematics',
 'statistics',
 'mathematics',
 'business',
 'business',
 'commerce',
 'chemistry',
 'engineering',
 'statistics',
 'physics',
 'marketing',
 'business',
 'statistics',
 'IT',
 'business',
 'IT',
 'mathematics',
 'physics',
 'chemistry',
 'business',
 'business',
 'business',
 'business',
 'business',
 'engineering',
 'business',
 'business',
 'IT',
 'engineering',
 'marketing',
 'business',
 'statistics',
 'business',


In [23]:
# dict to count degree types
deg_type_count = defaultdict(int)

for word in degree_type:
    deg_type_count[word] += 1
    
deg_type_count_sort = {k: v for k, v in sorted(deg_type_count.items(), key=lambda item: item[1], reverse=True)}
deg_type_count_sort

{'business': 115,
 'statistics': 65,
 'engineering': 62,
 'IT': 43,
 'mathematics': 37,
 'marketing': 14,
 'physics': 12,
 'chemistry': 9,
 'commerce': 6,
 'epidemiology': 1}

In [24]:
# dict to count degree levels
deg_level_count = defaultdict(int)

for l in degree_level:
    for word in l:
         deg_level_count[word] += 1
    
deg_level_count_sort = {k: v for k, v in sorted(deg_level_count.items(), key=lambda item: item[1], reverse=True)}
deg_level_count_sort

{'UNSPECIFIED': 541, 'bachelors': 195, 'masters': 32, 'phd': 10}

possible NMF?

## Part A: By Job Title

### i. Data Analyst

In [25]:
# STEPS:
# 1) head
# 2) histograms: job title, technologies, ratings heatmap? founded dates, type of company ownership, industry, sector, 
# company size, revenue, per region count for da,de,ds per country, 
# 3) *NMF job descr*# perform NNMF to find common phrases for jb descrip? (1 - general NMF for jd, 2) then for 
# skills/responsib etc.(subset jd first), 3) common degree types - extract string and NMF
# word clouds? DA DS DE tech?
# 
# 
# 
# 
# 
# 

### ii. Data Scientist

### iii. Data Engineer

### iv. Combined

## Part B: By Country

### i. South Africa

In [26]:
# STEPS:
# 1) head
# 2) histograms: job title, technologies, ratings heatmap? founded dates, type of company ownership, industry, sector, 
# company size, revenue, per region count for da,de,ds per country, 
# 3) *NMF job descr*# perform NNMF to find common phrases for jb descrip? (1 - general NMF for jd, 2) then for 
# skills/responsib etc.(subset jd first), 3) common degree types - extract string and NMF
# 
# 
# 
# 
# 
# 
# 

### ii. United States

### iii. United Kingdom

### iv. Australia

### v. Combined